In [5]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader
from tensorflow_examples.lite.model_maker.core.task import image_classifier
from tensorflow_examples.lite.model_maker.core.task.model_spec import efficientnet_lite4_spec
from tensorflow_examples.lite.model_maker.core.task.model_spec import ImageModelSpec

import matplotlib.pyplot as plt
# from matplotlib.pyplot import specgram
# import librosa
# import librosa.display

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [7]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pandas as pd

In [8]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
def confusion(y_true, y_pred, labels, perc = False):
    conf = confusion_matrix(y_true, y_pred, labels)
    conf = pd.DataFrame(conf, columns = labels)
    conf['true_row_label'] = labels
    conf.set_index('true_row_label', drop = True, inplace = True)
    if perc:
        conf = round(conf.div(conf.sum(axis=1), axis=0),2)
    return conf

# Multiple Models

In [1]:
def train_valid_folder_images(folder):
    folder = str(folder)
    base_path = "../downsampled/imagenet_structure/"
    return ImageClassifierDataLoader.from_folder(base_path + folder + "/train"), ImageClassifierDataLoader.from_folder(base_path + folder + "/valid")

In [2]:
def create_fit_submodel(image_folder_substring, epochs = 10, warmup_steps = 100, batch_size = 24):
    train_data, valid_data = train_valid_folder_images(image_folder_substring)
    return image_classifier.create(train_data, 
                                      model_spec=efficientnet_lite4_spec, 
                                      shuffle = True,
                                      epochs = epochs, 
                                      batch_size = batch_size,
                                      warmup_steps = warmup_steps, 
                                      validation_data = valid_data)

In [3]:
import os

In [4]:
def save_ensemble_model(model, path):
    if not os.path.exists(path):
        os.mkdir(path)
    model.model.save(path)
    model.export(path +'/model.tflite', path+'/labels.txt')

In [14]:
# import os
# import glob
# import shutil
# from pathlib import Path  

In [51]:

# data_path = Path('../downsampled/imagenet_structure/ensemble/') 
# submodels = {'engine-air-other': ['air_conditioner', 'engine_idling'], 
#              'drilling-jackhammer-other': ['drilling', 'jackhammer'] , 
#              'other': ['car_horn', 'children_playing', 'dog_bark', 'siren', 'street_music']}
# labels  = ['air_conditioner','car_horn','children_playing',
#            'dog_bark','drilling','engine_idling','gun_shot','jackhammer','siren','street_music']
# def move_submodel_files(submodel_folder_name, submodel_class_list):
#     if d in ['train', 'valid]':
#         print(d)
#         if not os.path.exists(data_path/submodel_folder_name/d/'other'):
#             os.mkdir(data_path/submodel_folder_name/d/'other')
#         for c in np.setdiff1d(labels,submodel_class_list,True).tolist():
#             png_files = list(Path(data_path/submodel_folder_name/d/c).glob('*.png'))
#             for f in png_files:
#                 shutil.move(str(f), str(data_path/submodel_folder_name/d/'other'))
#             os.rmdir(data_path/submodel_folder_name/d/c)

# for k, v in submodels.items():
#     print(k)
#     move_submodel_files(k,v)

## Main Model

In [13]:
model_main = create_fit_submodel('ensemble/main_model', epochs = 50)

INFO:tensorflow:Load image with size: 7859, num_label: 10, labels: air_conditioner, car_horn, children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren, street_music.
INFO:tensorflow:Load image with size: 873, num_label: 10, labels: air_conditioner, car_horn, children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren, street_music.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              11837936  
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 11,850,746
Trainable params: 12,810
Non-trainable params: 11,837,936
_________________________________________________________________
None
INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Train for 327 steps, validate for 36 steps
Epoch 1/50
327/327 [==============================] - 146s 445ms/step - loss: 1.4889 - accuracy: 0.5766 - val_loss: 1.2508 - val_accuracy: 0.6875
Epoch 2/50
327/327 [==============================] - 140s 428ms/step - loss: 1.2370 - accuracy: 0.6980 - val_loss: 1.1792 - val_accuracy: 0.7118
Epoch 3/50
327/327 [==============================] - 140s 429ms/step - loss: 1.1696 - accuracy: 0.7283 - val_loss: 1.1588 - val_accuracy: 0.7164
Epoch 4/50
327/327 [==============================] - 140s 429ms/step - loss: 1.1433 - accuracy: 0.7446 - val_loss: 1.1506 - val_accuracy: 0.7303
Epoch 5/50
327/327 [==============================] - 140s 429ms/step - loss: 1.1106 - accuracy: 0.7566 - val_loss: 1.1357 - val_accuracy: 0.7326
Epoch 6/50
327/327 [==============================] - 140s 429ms/step - loss: 1.0918 - accuracy: 0.7666 - val_loss: 1.1387 - val_accuracy: 0.7361
Epoch 7/50
327/327 [==============================] - 140s 429ms/step - loss: 1.0

In [23]:
model_name = 'main_model_e50'
os.mkdir('./models/ensemble/'+model_name)
model_main.model.save('models/ensemble/' + model_name)

INFO:tensorflow:Assets written to: models/ensemble/main_model_e50\assets


INFO:tensorflow:Assets written to: models/ensemble/main_model_e50\assets


In [24]:
model_path_prefix = './models/ensemble/'+ model_name 
model_main.export(model_path_prefix+'/model.tflite', model_path_prefix+'/labels.txt')

INFO:tensorflow:Export to tflite model in ./models/ensemble/main_model_e50/model.tflite.


INFO:tensorflow:Export to tflite model in ./models/ensemble/main_model_e50/model.tflite.


INFO:tensorflow:Saved labels in ./models/ensemble/main_model_e50/labels.txt.


INFO:tensorflow:Saved labels in ./models/ensemble/main_model_e50/labels.txt.


### Load and Test Saved Model

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def folder_data_generator(folder_path, 
                          batch_size = 24, 
                          input_image_shape = (300,300),
                          shuffle = True, 
                          seed = 42):
    """Generates an ImageDataGenerator that is scaled by (1/255) and
        resized to the required input shape.
    
    Args:
    
    folder_path: path to directory containing images organized in class-named subdirectories
        Example: '../downsampled/imagenet_structure/ensemble/model-main/train/'
    hparams (dataclas):  Hyperparameters with minimum requirements:
        .input_image_shape (tuple): image shape required by model, to be resized
        .batch_size (int): batch size for generator
    shuffe (bool): if images should be shuffled.  Defaults to True
    seed (int): seed for random shuffle. Defaults to 42
    
    Assumes class_mode = 'categorical' and color_mode = 'rgb' for images.
    
    Returns:
    
    ImageDataGenerator object
    """
    # ex: '../downsampled/imagenet_structure/1/valid/'
    datagen = ImageDataGenerator(rescale = 1.0/255.0)
    generator = datagen.flow_from_directory(
        directory = folder_path,
        target_size = input_image_shape, 
        class_mode = 'categorical', 
        color_mode = 'rgb',
        batch_size = batch_size,
        shuffle = shuffle, 
        seed = seed)
    return generator

## air-drill-engine-jackhammer

In [10]:
model_adej = create_fit_submodel('ensemble/air-drill-engine-jackhammer', epochs = 10)

INFO:tensorflow:Load image with size: 3584, num_label: 4, labels: air_conditioner, drilling, engine_idling, jackhammer.
INFO:tensorflow:Load image with size: 416, num_label: 4, labels: air_conditioner, drilling, engine_idling, jackhammer.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              11837936  
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 11,843,060
Trainable params: 5,124
Non-trainable params: 11,837,936
_________________________________________________________________
None
INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Train for 149 steps, validate for 17 steps
Epoch 1/10
149/149 [==============================] - 75s 502ms/step - loss: 1.0748 - accuracy: 0.5959 - val_loss: 0.9727 - val_accuracy: 0.6765
Epoch 2/10
149/149 [==============================] - 64s 432ms/step - loss: 0.9063 - accuracy: 0.7078 - val_loss: 0.9162 - val_accuracy: 0.7108
Epoch 3/10
149/149 [==============================] - 64s 428ms/step - loss: 0.8523 - accuracy: 0.7408 - val_loss: 0.8980 - val_accuracy: 0.7132
Epoch 4/10
149/149 [==============================] - 64s 427ms/step - loss: 0.8310 - accuracy: 0.7550 - val_loss: 0.8906 - val_accuracy: 0.7157
Epoch 5/10
149/149 [==============================] - 64s 427ms/step - loss: 0.8017 - accuracy: 0.7757 - val_loss: 0.9020 - val_accuracy: 0.6887
Epoch 6/10
149/149 [==============================] - 64s 427ms/step - loss: 0.7931 - accuracy: 0.7771 - val_loss: 0.8882 - val_accuracy: 0.6961
Epoch 7/10
149/149 [==============================] - 64s 427ms/step - loss: 0.7810 - a

In [19]:
save_ensemble_model(model_adej,'air-drill-engine-jackhammer_e10','./models/ensemble')

INFO:tensorflow:Assets written to: ./models/ensemble/air-drill-engine-jackhammer_e10\assets


INFO:tensorflow:Assets written to: ./models/ensemble/air-drill-engine-jackhammer_e10\assets


INFO:tensorflow:Export to tflite model in ./models/ensemble/air-drill-engine-jackhammer_e10/air-drill-engine-jackhammer_e10.tflite.


INFO:tensorflow:Export to tflite model in ./models/ensemble/air-drill-engine-jackhammer_e10/air-drill-engine-jackhammer_e10.tflite.


INFO:tensorflow:Saved labels in ./models/ensemble/air-drill-engine-jackhammer_e10/air-drill-engine-jackhammer_e10_labels.txt.


INFO:tensorflow:Saved labels in ./models/ensemble/air-drill-engine-jackhammer_e10/air-drill-engine-jackhammer_e10_labels.txt.


In [16]:
def valid_folder_images(folder):
    folder = str(folder)
    base_path = "../downsampled/imagenet_structure/"
    return ImageClassifierDataLoader.from_folder(base_path + folder + "/valid")

valid_data = valid_folder_images("ensemble/drilling-jackhammer-other")

INFO:tensorflow:Load image with size: 873, num_label: 3, labels: drilling, jackhammer, other.


INFO:tensorflow:Load image with size: 873, num_label: 3, labels: drilling, jackhammer, other.


In [17]:
valid_predicts = model_djo.predict_top_k(valid_data)
valid_label = [valid_data.index_to_label[label.numpy()] for i, (image, label) in enumerate(valid_data.dataset.take(len(valid_predicts)))]

In [18]:
valid_predict_label = [i[0][0] for i in valid_predicts]

In [ ]:
#e20
print(classification_report(y_true = valid_label, y_pred = valid_predict_label))

In [ ]:
confusion(valid_label, valid_predict_label, valid_data.index_to_label)

## horn-children-dog-gun-siren-music

In [12]:
group = 'horn-children-dog-gun-siren-music'

In [13]:
model_adej = create_fit_submodel('ensemble/' + group, epochs = 20)

INFO:tensorflow:Load image with size: 4275, num_label: 6, labels: car_horn, children_playing, dog_bark, gun_shot, siren, street_music.
INFO:tensorflow:Load image with size: 457, num_label: 6, labels: car_horn, children_playing, dog_bark, gun_shot, siren, street_music.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              11837936  
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 7686      
Total params: 11,845,622
Trainable params: 7,686
Non-trainable params: 11,837,936
_________________________________________________________________
None
INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Train for 178 steps, validate for 19 steps
Epoch 1/20
178/178 [==============================] - 85s 479ms/step - loss: 1.0856 - accuracy: 0.6821 - val_loss: 0.7710 - val_accuracy: 0.8487
Epoch 2/20
178/178 [==============================] - 76s 427ms/step - loss: 0.8684 - accuracy: 0.7980 - val_loss: 0.7486 - val_accuracy: 0.8531
Epoch 3/20
178/178 [==============================] - 76s 427ms/step - loss: 0.8226 - accuracy: 0.8287 - val_loss: 0.7283 - val_accuracy: 0.8772
Epoch 4/20
178/178 [==============================] - 76s 427ms/step - loss: 0.8047 - accuracy: 0.8397 - val_loss: 0.7304 - val_accuracy: 0.8728
Epoch 5/20
178/178 [==============================] - 76s 427ms/step - loss: 0.7869 - accuracy: 0.8497 - val_loss: 0.7269 - val_accuracy: 0.8662
Epoch 6/20
178/178 [==============================] - 76s 427ms/step - loss: 0.7756 - accuracy: 0.8481 - val_loss: 0.7167 - val_accuracy: 0.8772
Epoch 7/20
178/178 [==============================] - 77s 430ms/step - loss: 0.7712 - a

In [14]:
save_ensemble_model(model_adej, './models/ensemble/'+group)

INFO:tensorflow:Assets written to: ./models/ensemble/horn-children-dog-gun-siren-music\assets


INFO:tensorflow:Assets written to: ./models/ensemble/horn-children-dog-gun-siren-music\assets


INFO:tensorflow:Export to tflite model in ./models/ensemble/horn-children-dog-gun-siren-music/model.tflite.


INFO:tensorflow:Export to tflite model in ./models/ensemble/horn-children-dog-gun-siren-music/model.tflite.


INFO:tensorflow:Saved labels in ./models/ensemble/horn-children-dog-gun-siren-music/labels.txt.


INFO:tensorflow:Saved labels in ./models/ensemble/horn-children-dog-gun-siren-music/labels.txt.
